# Ensembles, Relations & Fonctions — Notebook interactif

**Objectifs**
- Manipuler les *ensembles* (union, intersection, différence, puissance, produit cartésien).
- Modéliser des *relations* (réflexive, symétrique, transitive) et les représenter.
- Travailler les *applications (fonctions)* : image, image réciproque, injectivité, surjectivité, bijectivité, composition, inverse.
- Réaliser des mini-challenges pour ancrer les concepts.

> Python sert ici d’outil de simulation pour tester vos intuitions mathématiques.


In [ ]:

# Imports (standard)
from itertools import product, permutations
import math

# Graphiques
import matplotlib.pyplot as plt

# Outils utilitaires
def powerset(s):
    # Retourne l'ensemble des parties (liste de frozenset).
    s = list(s)
    ps = []
    n = len(s)
    for mask in range(1<<n):
        subset = frozenset(s[i] for i in range(n) if (mask>>i) & 1)
        ps.append(subset)
    return ps

def cartesian(A, B):
    # Produit cartésien A×B (ensemble de couples).
    return {(a,b) for a in A for b in B}

def is_function(E, F, rel):
    # Vérifie si rel (ensemble de couples) définit une fonction E→F.
    if not all((x in E and y in F) for (x,y) in rel):
        return False
    # unicité de l'image
    seen = {}
    for x,y in rel:
        if x in seen and seen[x] != y:
            return False
        seen[x] = y
    # total : chaque x a une image
    return set(seen.keys()) == set(E)

def image(f, A):
    # Image directe f(A). f est un dict {x: y}.
    return {f[x] for x in A if x in f}

def preimage(f, B):
    # Image réciproque f^{-1}(B). f est un dict {x: y}.
    return {x for x,y in f.items() if y in B}

def is_injective(f, E=None, F=None):
    # Injectivité sur E→F. f est un dict {x:y} total sur E si E fourni.
    if E is not None and set(f.keys()) != set(E):
        return False
    vals = list(f.values())
    return len(vals) == len(set(vals))

def is_surjective(f, E, F):
    # Surjectivité sur F : l'image couvre F.
    return set(f[x] for x in E if x in f) == set(F)

def is_bijective(f, E, F):
    return is_injective(f, E, F) and is_surjective(f, E, F)

def compose(g, f):
    # Composition g∘f, f:X->Y, g:Y->Z, représentées par des dicts totaux.
    return {x: g[f[x]] for x in f}

def inverse_bijection(f):
    # Inverse si f est bijection (dict ↦ dict).
    inv = {}
    for x,y in f.items():
        if y in inv:
            raise ValueError("Pas injective")
        inv[y] = x
    return inv

def check_relation_properties(E, R):
    # Retourne dict des propriétés pour une relation R⊆E×E.
    ref = all((x,x) in R for x in E)
    sym = all(((y,x) in R) for (x,y) in R)
    trans = all(((x,z) in R) for (x,y) in R for (y2,z) in R if y2==y)
    anti_sym = all((x==y) or ((y,x) not in R) for (x,y) in R)
    return {"reflexive": ref, "symetrique": sym, "transitive": trans, "antisymetrique": anti_sym}



## 1. Ensembles (Sets)

Python possède un type `set` très pratique pour expérimenter les opérations ensemblistes.


In [ ]:

# Démonstration basique
A = {1,2,3,4}
B = {3,4,5}
U = set(range(1,8))  # "univers" ici

print("A =", A)
print("B =", B)
print("A ∪ B =", A | B)
print("A ∩ B =", A & B)
print("A \\ B =", A - B)
print("B \\ A =", B - A)

# Complément dans U (par rapport à notre univers choisi)
Ac = U - A
print("Complément de A (dans U) =", Ac)

# Produit cartésien
AxB = cartesian(A, B)
print("|A×B| =", len(AxB), "Ex:", list(AxB)[:5])

# Ensemble des parties
P = powerset({1,2,3})
print("℘({1,2,3}) a", len(P), "éléments. Exemples:", sorted([sorted(list(s)) for s in P])[:6])



### Visualisation rapide (zone qualitative, *façon Venn*)

On trace deux disques qui se recouvrent pour visualiser (A ∩ B). *Illustratif*, pas à l’échelle.


In [ ]:

# Venn "maison" (sans bibliothèques externes)
fig = plt.figure(figsize=(5,4))
ax = plt.gca()
circle1 = plt.Circle((0.0, 0), 1.0, alpha=0.3)
circle2 = plt.Circle((0.8, 0), 1.0, alpha=0.3)
ax.add_patch(circle1); ax.add_patch(circle2)
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(-1.5, 2.0); ax.set_ylim(-1.2, 1.2)
ax.axis('off')
ax.text(-0.8, 0.9, "Univers U", fontsize=10)
ax.text(-0.4, 0.0, "A")
ax.text(0.9, 0.0, "B")
plt.show()



### 🧩 Challenge — Ensembles
1. Construisez deux ensembles `X` et `Y` de votre choix (5–8 éléments).  
2. Calculez `X∪Y`, `X∩Y`, `X\Y` et le produit cartésien `X×Y`.  
3. Écrivez une fonction qui teste la loi de De Morgan :  
   \[(X\cup Y)^c = X^c \cap Y^c\] par rapport à un univers `U` donné.


In [ ]:

# Espace de travail — à vous !
U = set(range(1,21))
X = {1,3,5,7,9,12}
Y = {2,3,6,7,10,12,14}

X_union_Y = X | Y
X_inter_Y = X & Y
X_minus_Y = X - Y
XY = cartesian(X, Y)

print("X∪Y:", X_union_Y)
print("X∩Y:", X_inter_Y)
print("X\\Y:", X_minus_Y)
print("|X×Y|:", len(XY))

def demorgan(U, X, Y):
    lhs = (U - (X | Y))
    rhs = (U - X) & (U - Y)
    return lhs == rhs

print("De Morgan vérifiée ?", demorgan(U, X, Y))



## 2. Relations

Une relation \(R \subseteq E\times E\) peut être représentée comme un ensemble de couples `(x,y)`.
Propriétés classiques : **réflexive**, **symétrique**, **transitive**, **antisymétrique**.


In [ ]:

# Exemple : relation "divise" sur E={1,2,3,4,6}
E = {1,2,3,4,6}
R = {(a,b) for a in E for b in E if b % a == 0}
props = check_relation_properties(E, R)
print("Propriétés de 'divise' :", props)

# Autre relation : "même parité"
Rp = {(a,b) for a in E for b in E if (a-b) % 2 == 0}
props_p = check_relation_properties(E, Rp)
print("Propriétés 'même parité' :", props_p)



### Visualisation : matrice d’adjacence

On affiche une "carte" (x→y) pour voir les couples présents dans la relation.


In [ ]:

E_sorted = sorted(E)
Rmap = {(x,y): ((x,y) in R) for x in E_sorted for y in E_sorted}

fig = plt.figure(figsize=(4,4))
ax = plt.gca()
for i,x in enumerate(E_sorted):
    for j,y in enumerate(E_sorted):
        if Rmap[(x,y)]:
            ax.plot(j, len(E_sorted)-1-i, marker='s')
        else:
            ax.plot(j, len(E_sorted)-1-i, marker='.', alpha=0.2)
ax.set_xticks(range(len(E_sorted))); ax.set_yticks(range(len(E_sorted)))
ax.set_xticklabels([str(y) for y in E_sorted])
ax.set_yticklabels([str(x) for x in reversed(E_sorted)])
ax.set_xlabel('y'); ax.set_ylabel('x')
ax.set_title("Matrice d'adjacence de R")
plt.show()



### 🧩 Challenge — Relations
1. Construisez une relation \(R\) sur \(E=\{1,2,3,4,5\}\) qui soit :  
   (a) réflexive et symétrique mais pas transitive,  
   (b) réflexive et transitive mais pas symétrique.  
2. Écrivez un test qui décide si \(R\) est une **relation d’équivalence** puis, si oui, calculez ses **classes d’équivalence**.


In [ ]:

def is_equivalence(E, R):
    p = check_relation_properties(E, R)
    return p["reflexive"] and p["symetrique"] and p["transitive"]

def equivalence_classes(E, R):
    # Retourne la liste des classes [x] si R est une équivalence.
    if not is_equivalence(E, R):
        return None
    classes = []
    seen = set()
    for x in E:
        if x in seen: 
            continue
        cl = {y for y in E if (x,y) in R and (y,x) in R}
        classes.append(cl)
        seen |= cl
    return classes

# Démo avec la relation "même parité" Rp
print("Rp est équivalence ?", is_equivalence(E, Rp))
print("Classes :", equivalence_classes(E, Rp))



## 3. Fonctions (Applications)

On modélise une application \(f:E\to F\) par un **dictionnaire** Python `{x: y}` total sur `E`.
On traite : image, image réciproque, injectivité, surjectivité, bijection, composition, inverse.


In [ ]:

# Domaine et codomaine
E = {1,2,3,4}
F = {"a","b","c"}

# Exemple 1 : pas surjective (image = {"a","b"})
f = {1:"a", 2:"b", 3:"a", 4:"b"}
print("f est fonction E→F ?", set(f.keys())==E and set(f.values()).issubset(F))

print("Image de {1,3} :", image(f, {1,3}))
print("Préimage de {'a'} :", preimage(f, {"a"}))

print("Injective ?", is_injective(f, E, F))
print("Surjective ?", is_surjective(f, E, F))
print("Bijective ?", is_bijective(f, E, F))

# Exemple 2 : bijection E→E
E2 = {1,2,3,4}
f2 = {1:3, 2:1, 3:4, 4:2}
print("\nBijection ? (E2→E2)", is_bijective(f2, E2, E2))
g2 = inverse_bijection(f2)
print("Inverse f2^{-1}:", g2)
print("Composition g2∘f2 = id ?", compose(g2, f2)=={x:x for x in E2})



### Visualisation : graphe des flèches (E → F)

Représentation simple des flèches d’une application finie.


In [ ]:

def plot_function_arrows(E, F, f):
    # positions simples sur deux lignes
    Ex = sorted(list(E))
    Fx = sorted(list(F), key=lambda x: str(x))
    fig = plt.figure(figsize=(6,2.5))
    ax = plt.gca()
    for i,x in enumerate(Ex):
        ax.plot(i, 1, marker='o')
        ax.text(i, 1.1, str(x), ha='center', va='bottom', fontsize=10)
    for j,y in enumerate(Fx):
        ax.plot(j, 0, marker='o')
        ax.text(j, -0.1, str(y), ha='center', va='top', fontsize=10)
    # flèches
    for i,x in enumerate(Ex):
        y = f.get(x, None)
        if y is None: 
            continue
        j = Fx.index(y)
        ax.annotate("", xy=(j, 0.08), xytext=(i, 0.92), arrowprops=dict(arrowstyle="->"))
    ax.set_xlim(-0.5, max(len(Ex), len(Fx)) - 0.5)
    ax.set_ylim(-0.5, 1.5)
    ax.axis('off')
    ax.set_title("f : E → F")
    plt.show()

plot_function_arrows(E, F, f)



### 🧩 Challenge — Fonctions
1. Construisez une application `h:E→F` sur des ensembles finis de votre choix.  
2. Testez l’injectivité, la surjectivité et la bijectivité.  
3. Si `h` est bijection, calculez `h^{-1}` et vérifiez `h^{-1}∘h = id`.
4. Construisez deux applications `f:X→Y` et `g:Y→Z` et codez la composition `g∘f`.


In [ ]:

# Espace de travail — à vous !
X = {"x1","x2","x3"}
Y = {"y1","y2","y3"}
Z = {"z1","z2","z3"}

f = {"x1":"y2", "x2":"y3", "x3":"y1"}
g = {"y1":"z3", "y2":"z1", "y3":"z2"}

print("Injective f ?", is_injective(f, X, Y))
print("Surjective f ?", is_surjective(f, X, Y))
print("Bijective f ?", is_bijective(f, X, Y))

h = compose(g, f)
print("g∘f =", h)

plot_function_arrows(X, Z, h)



## Auto-checks rapides (optionnels)

Quelques fonctions de vérification pour vos réponses aux challenges.


In [ ]:

def check_demorgan(U, X, Y):
    return (U - (X | Y)) == ((U - X) & (U - Y))

def check_equivalence(E, R):
    return is_equivalence(E, R), equivalence_classes(E, R)

def check_function_props(E, F, f):
    return {
        "function?": set(f.keys())==set(E) and set(f.values()).issubset(F),
        "injective": is_injective(f, E, F),
        "surjective": is_surjective(f, E, F),
        "bijective": is_bijective(f, E, F)
    }

# Exemple d'usage:
# check_demorgan(U, X, Y)
# check_equivalence(E, Rp)
# check_function_props(E, F, f)



## Wrap-up

- Les **ensembles** se manipulent naturellement en Python (`set`).
- Les **relations** s’implémentent comme des sous-ensembles de `E×E` ; propriétés testées par fonctions dédiées.
- Les **fonctions** se codent comme des dictionnaires `f:E→F` : image, préimage, composition, inverse (si bijection).

> Prochain pas : *Ensembles infinis, relations d’ordre, applications partielles et restrictions*, ainsi que *décomptes* et *principe des tiroirs* en contexte de fonctions.
